In [31]:
# import necessary libraries

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [32]:
# setup browser and url

browser = webdriver.Chrome()

url = 'https://footystats.org'

In [33]:
# open website

browser.get(url)
browser.maximize_window()

# wait until dropdown menu is loaded
WebDriverWait(browser, 20).until(
    EC.presence_of_element_located((By.XPATH, "//*[text()='Leagues']"))
)

<selenium.webdriver.remote.webelement.WebElement (session="7734e42b580873d2a0ebbf6b26bdd6b8", element="f.FE1F4233904D9D3C7452156D8182ED2E.d.2FF9500CDAC4C0A1C114441A4D54D355.e.328")>

In [34]:
# grab links to chosen leagues from dropdown menu

leagues = ['Premier League', 'La Liga', 'Ligue 1', 'Bundesliga', 'Serie A']

# hover over 'Leagues' dropdown menu
leagues_dropdown = browser.find_element(By.XPATH, "//*[text()='Leagues']")
webdriver.ActionChains(browser).move_to_element(leagues_dropdown).perform()

# find and extract links to chosen leagues from dropdown menu
leagues_links = []
for league in leagues:
    league_link = browser.find_element(By.XPATH, f"//*[text()='{league}']")
    leagues_links.append(league_link.get_attribute('href'))
    
# print out for debugging
for league, link in zip(leagues, leagues_links):
    print(league, link)

Premier League https://footystats.org/england/premier-league
La Liga https://footystats.org/spain/la-liga
Ligue 1 https://footystats.org/france/ligue-1
Bundesliga https://footystats.org/germany/bundesliga
Serie A https://footystats.org/italy/serie-a


### Take all links of clubs from league links

In [35]:
import numpy as np

all_club_links = []

for link in leagues_links:
    browser.get(link)
    WebDriverWait(browser, 20).until(EC.presence_of_element_located((By.CLASS_NAME, "table-wrapper")))
    club_elements = browser.find_elements(By.XPATH, "//table//tr//td//a[contains(@href, '/clubs/')]")
    club_links = np.array([club_link.get_attribute('href') for club_link in club_elements])
    all_club_links.extend(club_links)

all_club_links = np.array(list(set(all_club_links)))

print(len(all_club_links))

for link in all_club_links:
    print(link)

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF755E3B095+29557]
	(No symbol) [0x00007FF755DAFA50]
	(No symbol) [0x00007FF755C6B56A]
	(No symbol) [0x00007FF755CBF695]
	(No symbol) [0x00007FF755CBF8EC]
	(No symbol) [0x00007FF755D0B777]
	(No symbol) [0x00007FF755CE71CF]
	(No symbol) [0x00007FF755D0851C]
	(No symbol) [0x00007FF755CE6F33]
	(No symbol) [0x00007FF755CB116F]
	(No symbol) [0x00007FF755CB22D1]
	GetHandleVerifier [0x00007FF75616C96D+3378253]
	GetHandleVerifier [0x00007FF7561B8497+3688311]
	GetHandleVerifier [0x00007FF7561AD1CB+3642539]
	GetHandleVerifier [0x00007FF755EFA6B6+813462]
	(No symbol) [0x00007FF755DBAB5F]
	(No symbol) [0x00007FF755DB6B74]
	(No symbol) [0x00007FF755DB6D10]
	(No symbol) [0x00007FF755DA5C1F]
	BaseThreadInitThunk [0x00007FFBE11B7AC4+20]
	RtlUserThreadStart [0x00007FFBE208A8C1+33]


### Take all links of players of 5 leagues

In [ ]:
all_players_link = []

for link in all_club_links:
    browser.get(link)
    WebDriverWait(browser, 20).until(EC.presence_of_element_located((By.XPATH, "//*[text()='Players']")))
    browser.find_element(By.XPATH, "//*[text()='Players']").click()
    players_elements = browser.find_elements(By.CLASS_NAME, "semi-bold")
    players_links = np.array([player_link.get_attribute('href') for player_link in players_elements])
    all_players_link.extend(players_links)

all_players_link = np.array(list(set(all_players_link)))

print(len(all_players_link))

# for link in all_players_link:
#     print(link)

TimeoutException: Message: timeout: Timed out receiving message from renderer: 299.513
  (Session info: chrome=129.0.6668.101)
Stacktrace:
	GetHandleVerifier [0x00007FF755E3B095+29557]
	(No symbol) [0x00007FF755DAFA50]
	(No symbol) [0x00007FF755C6B56A]
	(No symbol) [0x00007FF755C541E7]
	(No symbol) [0x00007FF755C53EEA]
	(No symbol) [0x00007FF755C51D65]
	(No symbol) [0x00007FF755C525BF]
	(No symbol) [0x00007FF755C60F4E]
	(No symbol) [0x00007FF755C79A1D]
	(No symbol) [0x00007FF755C7F5BA]
	(No symbol) [0x00007FF755C52D24]
	(No symbol) [0x00007FF755C79879]
	(No symbol) [0x00007FF755D08861]
	(No symbol) [0x00007FF755CE6F33]
	(No symbol) [0x00007FF755CB116F]
	(No symbol) [0x00007FF755CB22D1]
	GetHandleVerifier [0x00007FF75616C96D+3378253]
	GetHandleVerifier [0x00007FF7561B8497+3688311]
	GetHandleVerifier [0x00007FF7561AD1CB+3642539]
	GetHandleVerifier [0x00007FF755EFA6B6+813462]
	(No symbol) [0x00007FF755DBAB5F]
	(No symbol) [0x00007FF755DB6B74]
	(No symbol) [0x00007FF755DB6D10]
	(No symbol)

In [ ]:
import re
# 
def get_stat_number_int(stat):
    stat_number = int(re.findall(r'\d+', stat)[0])
    return stat_number

def get_stat_number_float(stat):
    stat_number_match = re.findall(r'\d+', stat)
    stat_number = float(stat_number_match[0] + "." + stat_number_match[1])
    return stat_number

In [ ]:
# Set up the Selenium WebDriver (make sure you have installed the required driver for your browser, e.g., ChromeDriver)
browser = webdriver.Chrome()

for link in all_players_link:
    # Open the page
    browser.get(link)

    # Wait until the div with id 'h2h_content3' is present
    content_div = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "h2h_content3"))
    )
    # Locate the div with id 'h2h_content1'
    content_div = browser.find_element(By.ID, "h2h_content1")
    name = (content_div.find_element(By.XPATH, './/div[@class="w100 row cf"]/h1[@class="row white heroh1"]').text).split(" Stats")[0].strip() 
    age = (content_div.find_element(By.XPATH, './/div[@class="fl rw33 white fs08e pb05e mr15e"]/span[@class="semi-bold"]').text).split('(')[0].strip()
    nationality = content_div.find_element(By.XPATH, './/div[@class="fl rw33 white fs08e ml15e"][1]/span[@class="semi-bold"]').text
    club = content_div.find_element(By.XPATH, './/div[@class="fl rw50 white fs08e"]/a').text
    position = content_div.find_element(By.XPATH, './/div[@class="fl rw33 white fs08e"][1]/span[@class="semi-bold"]').text
    height = get_stat_number_int(content_div.find_element(By.XPATH, './/div[@class="fl rw33 white fs08e mr15e"][1]/span[@class="semi-bold"]').text)
    weight = get_stat_number_int(content_div.find_element(By.XPATH, './/div[@class="fl rw33 white fs08e mr15e"][2]/span[@class="semi-bold"]').text)
    foot = content_div.find_element(By.XPATH, './/div[@class="fl rw33 white fs08e ml15e"][2]/span[@class="semi-bold"]').text


    content_div = browser.find_element(By.ID, "h2h_content2")
    total_matches = content_div.find_element(By.XPATH, './/td[contains(text(), "Matches Played")]/following-sibling::td[1]').text
    total_minutes = content_div.find_element(By.XPATH, './/td[contains(text(), "Minutes")]/following-sibling::td[1]').text
    total_goals = content_div.find_element(By.XPATH, './/td[contains(text(), "Goals Scored")]/following-sibling::td[1]').text
    total_assists = content_div.find_element(By.XPATH, './/td[contains(text(), "Assists")]/following-sibling::td[1]').text
    total_yellow = content_div.find_element(By.XPATH, './/td[contains(text(), " Yellow Cards")]/following-sibling::td[1]').text
    total_red = content_div.find_element(By.XPATH, './/td[contains(text(), " Red Cards")]/following-sibling::td[1]').text
    minutes_per_goal = get_stat_number_int(content_div.find_element(By.XPATH, './/td[contains(text(), "Minutes Per Goal")]/following-sibling::td[1]').text)
    shot_accuracy = get_stat_number_float(content_div.find_element(By.XPATH, './/td[contains(text(), "Shot Accuracy")]/following-sibling::td[1]').text)
    pass_completion_rate = get_stat_number_float(content_div.find_element(By.XPATH, './/td[contains(text(), "Pass Completion Rate")]/following-sibling::td[1]').text)
    cross_completion_rate = get_stat_number_float(content_div.find_element(By.XPATH, './/td[contains(text(), "Cross Completion Rate")]/following-sibling::td[1]').text)
    minutes_per_assist = get_stat_number_int(content_div.find_element(By.XPATH, './/td[contains(text(), "Minutes Per Assist")]/following-sibling::td[1]').text)
    dribble_success_rate = get_stat_number_float(content_div.find_element(By.XPATH, './/td[contains(text(), "Dribble Success Rate")]/following-sibling::td[1]').text)
    penalty_conversion_rate = content_div.find_element(By.XPATH, './/td[contains(text(), "Penalty Conversion Rate")]/following-sibling::td[1]').text
    penalty_conversion_rate_match = re.findall(r'\d+', penalty_conversion_rate)
    if (penalty_conversion_rate_match):
        penalty_conversion_rate = get_stat_number_float(penalty_conversion_rate)
    tackles = content_div.find_element(By.XPATH, './/td[contains(text(), "Tackles")]/following-sibling::td[1]').text
    interception = content_div.find_element(By.XPATH, './/td[contains(text(), "Interceptions")]/following-sibling::td[1]').text


    content_div = browser.find_element(By.ID, "h2h_content3")
    market_value_section = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//div[@class="title-bar"]'))
    )
    market_value = get_stat_number_float(content_div.find_element(By.XPATH, './/h3').text)
    titles = get_stat_number_int(content_div.find_element(By.XPATH, './/p[@class="mt1e lh14e header-blue semi-bold fs09e"]').text)

# print("Name:", name)
# print("Age:", age)
# print("Nationality:", nationality)
# print("Club:", club)
# print("Position:", position)
# print("Height:", height)
# print("Weight:", weight)
# print("Foot:", foot)
# print("Total Matches Played:", total_matches)
# print("Total Minutes:", total_minutes)
# print("Total Goals Scored:", total_goals)
# print("Total Assists:", total_assists)
# print("Total Yellow Cards:", total_yellow)
# print("Total Red Cards:", total_red)
# print("Minutes Per Goal:", minutes_per_goal)
# print("Shot Accuracy:", shot_accuracy)
# print("Pass Completion Rate:", pass_completion_rate)
# print("Cross Completion Rate:", cross_completion_rate)
# print("Minutes Per Assist:", minutes_per_assist)
# print("Dribble Success Rate:", dribble_success_rate)
# print("Penalty Conversion Rate:", penalty_conversion_rate)
# print("Tackles:", tackles)
# print("Interceptions:", interception)
# print("Market Value:", market_value)
# print("Titles Won:", titles)


# Close the browser
driver.quit()


Name: Kai Havertz
Age: 25
Nationality: Germany
Club: Arsenal
Position: Midfielder - Attacking Midfield
Height: 193
Weight: 83
Foot: Both
Total Matches Played: 7
Total Minutes: 630
Total Goals Scored: 4
Total Assists: 1
Total Yellow Cards: 0
Total Red Cards: 0
Minutes Per Goal: 158
Shot Accuracy: 52.63
Pass Completion Rate: 76.43
Cross Completion Rate: 33.33
Minutes Per Assist: 630
Dribble Success Rate: 31.58
Penalty Conversion Rate: No Pens
Tackles: 7
Interceptions: 4
Market Value: 70.0
Titles Won: 6


In [ ]:
# quit the browser

browser.quit()